# Table of Contents
* [Load Data](#section-1)
* [Data Exploration & Cleaning](#section-2)



# Load Data

In [1]:
import pandas as pd
import os 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
#Combining all the years together 
# data came from here - https://pge-energydatarequest.com/public_datasets/download?type=electric&file=PGE_2021_Q4_ElectricUsageByZip.zip
# Create an empty list to store the dataframes
current_dir = os.getcwd()
df_list = []

# Loop through each CSV file in the directory
for filename in os.listdir(current_dir):
    if filename.endswith('.csv'):
        # Read the CSV file and append it to the list of dataframes
        file_path = os.path.join(current_dir, filename)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.lower()
        #df['Quarter']=filename[9:11]
        df_list.append(df)

# Concatenate all dataframes in the list into a single dataframe
df_years = pd.concat(df_list, ignore_index=True)

ValueError: No objects to concatenate

# Data Exploration & Cleaning

In [ ]:
df_years

In [ ]:
df_years.columns

In [ ]:
df_years.info() 
# some columns like total customers are not integer or float types we'll need to fix this

In [ ]:
df_years['totalcustomers']=df_years['totalcustomers'].str.replace(',','')
df_years['totalkwh']=df_years['totalkwh'].str.replace(',','')
df_years['averagekwh']=df_years['averagekwh'].str.replace(',','')

In [ ]:
df_years['totalcustomers']=df_years['totalcustomers'].astype(int)
df_years['totalkwh']=df_years['totalkwh'].astype(float)
df_years['averagekwh']=df_years['averagekwh'].astype(float)

In [ ]:
df_years.describe()

In [ ]:
df_years['combined'].value_counts(normalize=True)*100 # 65% of our data belongs to combines zip codes

In [ ]:
df_years.duplicated().any() #determine duplicates

In [ ]:
df_years.duplicated().sum() # we have 14863 duplicates 

In [ ]:
df_years[df_years.duplicated(keep=False)].sort_values(by=['zipcode','month','year']) # lets see these duplicates ourselves

In [ ]:
df_years=df_years.drop_duplicates()

In [ ]:
df_years

In [ ]:
df_years['customerclass'].value_counts() # we have 4 classess of customers

In [ ]:
#lets randomly check data consistency 
df_years[(df_years['zipcode']==94538) & (df_years['year']==2014)]['month'].value_counts()
# we see that in 2014 we have 6 observations per month which is not what we expected 
# since we only have 4 types of customers

In [ ]:
#lets look closer
df_years[(df_years['zipcode']==94538) & (df_years['year']==2014)].sort_values(by=['customerclass','combined','month'])
# we see that for some zipcodes some months are repeated but with different number of total customers
# since we don't expect one zipcode to appear in the data several times per month 
# we should drop these duplicates 

In [ ]:
df_years[df_years.duplicated(subset=['zipcode', 'combined','month','year','customerclass','totalcustomers'], keep=False)].sort_values(by=['zipcode','month','year'])
# these are our near duplicates

In [ ]:
len(df_years[df_years.duplicated(subset=['zipcode', 'combined','month','year','customerclass','totalcustomers'])])
# we have 1520 such near duplicates 


In [ ]:
df_years=df_years.drop_duplicates(subset=['zipcode', 'combined','month','year','customerclass','totalcustomers'], keep='first')
# lets keep the first occurence 
#later we can play with what happens if we keep the secon occurence
# or create a measure which keeps the average of totalkwh and averagekwh  across all occurences

In [ ]:
len(df_years[(df_years['totalcustomers']==0) & (df_years['totalkwh']==0)& (df_years['averagekwh'].isnull())])
# We also have some lines with all 0s in total customers, total kilowatt per hour and 
# missing average kilo watt per hour 
# this may indicate zip codes where PGE does not have any customers / no coverage 
# Since these observations does not add any info except the zipcodes themselves we could drop them 

In [ ]:
mask=(df_years['totalcustomers']==0) & (df_years['totalkwh']==0) & (df_years['averagekwh'].isnull())
# filtering for these observations 

df_years=df_years[~mask] # excluding these observations from our dataset

In [ ]:
len(df_years[df_years.duplicated(subset=['zipcode', 'combined','month','year','customerclass'])])
# lets check for other duplicates based on 'zipcode', 'combined','month','year','customerclass' only - we have 7777 of them

In [ ]:
df_years[df_years.duplicated(subset=['zipcode', 'combined','month','year','customerclass'],keep=False)].sort_values(by=['zipcode','month','year'])

In [ ]:
df_years=df_years.drop_duplicates(subset=['zipcode', 'combined','month','year','customerclass'], keep='first')
# lets drop these duplicates too and keep the firts occurence

In [ ]:
df_years[(df_years['zipcode']==94538) & (df_years['year']==2014)].sort_values(by=['customerclass','combined','month'])
# now the data looks good 

In [ ]:
df_years[(df_years['customerclass']=='Elec- Commercial') & (df_years['year']==2013)& (df_years['zipcode']==94538) ]['month'].value_counts()
# keep checking the data 
# this looks good as we have one observation for this zip code per commercial customer class 

In [ ]:
df_years[(df_years['customerclass']=='Elec- Residential') & (df_years['year']==2014)& (df_years['zipcode']==93203) ]['month'].value_counts()


In [ ]:
df_years['customerclass'].value_counts() # we still have 4 classess of customers but not so many industrial and agricultural

In [ ]:
df_years['customerclass'].unique()

In [ ]:
df_years['zipcode'].nunique() # we have 827 zip codes here


In [ ]:
ohe = pd.get_dummies(df_years['customerclass'], drop_first=True) # we create 3 dummies for further analysis

In [ ]:
# Drops categorical variables from the df
#df_years = df_years.drop('customerclass', axis = 1) # lets not drop the initial avriable just for now

# Adds the newly created dummy variables instead
df_years = pd.concat([df_years, ohe], axis = 1) 

In [ ]:
df_years

In [ ]:
#determine if there is missing values
df_years.isna().any()

In [ ]:
df_years

# Data Visualization 

In [ ]:
def viz(customer_class, parameter):
    df=df_years[df_years['customerclass']==customer_class]
    unique_years = df['year'].unique()

    # Set the seaborn style
    sns.set(style='whitegrid')

    # Create a separate histogram for each year
    for year in unique_years:
        # Filter the DataFrame for the current year
        df_year = df[df['year'] == year]

        # Create a FacetGrid for the current year
        g = sns.FacetGrid(df_year, row='customerclass', height=3, aspect=3,sharex=False, sharey=False)
        g.map(sns.histplot, parameter, bins=200)

        # Set axis labels
        g.set_axis_labels(parameter, 'Count')

        for ax in g.axes.flat: # setting x and y to intersect at 0
            ax.set_xlim(left=0)
            ax.set_ylim(bottom=0)
        g.set_axis_labels(parameter, 'Count')

        for ax, customer_class in zip(g.axes.flat, g.row_names):
        # Extract part of the customer_class starting from index 6
            customer_class = customer_class[6:]
      
        for ax, title in zip(g.axes.flat, g.row_names):
            ax.set_title(f"Distribution of {customer_class} customers {parameter}: {year} year")
        plt.show()

In [ ]:
#def viz_med(parameter,year=None):
    grouped_data = df_years.groupby(['year', 'customerclass'])[parameter].median().reset_index()
    # Set the seaborn style
    sns.set(style='whitegrid')
    labels=['Residential', 'Commercial', 'Agricultural', 'Industrial']
    color_palette = sns.color_palette("flare", len(grouped_data['year'].unique()))

    # Plot the bar chart
    ax = sns.barplot(x='customerclass', y=parameter, hue='year', palette=color_palette,data=grouped_data,order=['Elec- Residential','Elec- Commercial','Elec- Agricultural',  'Elec- Industrial'])

    ax.set_xticklabels(labels)

    # Add labels and title
    plt.xlabel('Customer Class')
    plt.ylabel(f'Median Number of {parameter}')
    plt.title(f'Annual Median Number of {parameter} per Customer Class')

    plt.show()

In [ ]:
def viz_med(parameter,year=None):
    if year: # if year is given
        grouped_data = df_years[df_years['year'] == year].groupby(['year', 'customerclass'])[parameter].median().reset_index()
        title = f'Median Number of {parameter} per Customer Class for {year}'
    else:
        grouped_data = df_years.groupby(['year', 'customerclass'])[parameter].median().reset_index()
        title = f'Annual Median Number of {parameter} per Customer Class'
    # Set the seaborn style
    sns.set(style='whitegrid')
    labels=['Residential', 'Commercial', 'Agricultural', 'Industrial']
    color_palette = sns.color_palette("flare", len(grouped_data['year'].unique()))

    # Plot the bar chart
    ax = sns.barplot(x='customerclass', y=parameter, hue='year', palette=color_palette,data=grouped_data,order=['Elec- Residential','Elec- Commercial','Elec- Agricultural',  'Elec- Industrial'])

    ax.set_xticklabels(labels)

    # Add labels and title
    plt.xlabel('Customer Class')
    plt.ylabel(f'Median Number of {parameter}')
    plt.title(title)

    plt.show()

In [ ]:
viz('Elec- Residential','totalcustomers')

In [ ]:
viz('Elec- Commercial','totalcustomers')

In [ ]:
viz('Elec- Industrial','totalcustomers')

In [ ]:
viz('Elec- Agricultural','totalcustomers')

In [ ]:
viz_med('totalcustomers',year=2015)

In [ ]:
viz('Elec- Residential','totalkwh')

## Distribution of Monthly Average Consumption by years and customer classess

In [ ]:
viz('Elec- Residential','averagekwh')

In [ ]:
viz('Elec- Commercial','averagekwh')

In [ ]:
viz('Elec- Industrial','averagekwh')

In [ ]:
viz_med('averagekwh') #  industrial customers consumer a lot of kwh thats why the scale is in 10**6 units

In [ ]:
# lets graph median average kilowatt hours consumption wihout industrial clients
df_years_noind=df_years[df_years['customerclass']!='Elec- Industrial']
grouped_data = df_years.groupby(['year', 'customerclass'])['averagekwh'].median().reset_index()

# Set the seaborn style
sns.set(style='whitegrid')
labels=['Residential', 'Commercial', 'Agricultural']
color_palette = sns.color_palette("flare", len(grouped_data['year'].unique()))

# Plot the bar chart
ax = sns.barplot(x='customerclass', y='averagekwh', hue='year', palette=color_palette,data=grouped_data,order=['Elec- Residential','Elec- Commercial','Elec- Agricultural'])

ax.set_xticklabels(labels)

# Add labels and title
plt.xlabel('Customer Class')
plt.ylabel('Number of Customers')
plt.title('Annual Median Average kwh consumption per Customer Class')

plt.show()
# we see that residential customers consumer quite similar amount of kwh every year
# while there has been an increase in consumption in 2021 for Agricultural and Commercial Custoemrs

## Monthly and Yearly Trends Analysis

In [ ]:
# Lets explore if there is any seasonality in energy consumption 
# by plotting consumption over months

In [ ]:
import calendar

grouped_data = df_years.groupby(['customerclass', 'month']).agg({'averagekwh': 'median'})
# I use median as it's more robust to outliers as compared to average
grouped_data = grouped_data.reset_index()

# use all customer classes but industrial for which the consumption is very high and which we will plot separately
customer_classes = ['Elec- Residential','Elec- Commercial','Elec- Agricultural']
customer_class_labels = ['Residential Customers', 'Commercial Customers', 'Agricultural Customers']

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Iterate over each unique customer class and plot a line
for i,customer_class in enumerate(customer_classes):
    # Select data for the current customer class
    data = grouped_data[grouped_data['customerclass'] == customer_class]
    
    # Find the maximum average consumption (averagejwh) point in the data for the current customer class
    highest_point = data[data['averagekwh'] == data['averagekwh'].max()]
    
    # Plot the line for the current customer class
    ax.plot(data['month'], data['averagekwh'], label=customer_class_labels[i])
    
    # Plot a marker at the highest point
    ax.scatter(highest_point['month'], highest_point['averagekwh'], marker='o', color='red')
    
    # Customize x-axis labels with month names
    ax.set_xticks(data['month'])
    ax.set_xticklabels([calendar.month_name[m] for m in data['month']], rotation=45)

# Customize the plot labels and title
ax.set_xlabel("Month")
#ax.set_ylabel("Median of Average Kilowatt per hour energy consumption")
ax.set_title("Monthly Energy Consumption, kWh per Customer Average (2013-2021)")
ax.legend()
plt.show()
# looks like summer months - July & August are peak months 

In [ ]:
# plotting industrial customers
grouped_data = df_years.groupby(['customerclass', 'month']).agg({'averagekwh': 'median'})
# I use median as it's more robust to outliers as compared to average
grouped_data = grouped_data.reset_index()

# use all customer classes but industrial for which the consumption is very high and which we will plot separately
customer_classes = ['Elec- Industrial']
# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 5))
# Select data for the current customer class
data = grouped_data[grouped_data['customerclass'] == customer_class]
# Find the maximum average consumption (averagejwh) point 
highest_point = data[data['averagekwh'] == data['averagekwh'].max()]
ax.plot(data['month'], data['averagekwh'])
   
# Plot a marker at the highest point
ax.scatter(highest_point['month'], highest_point['averagekwh'], marker='o', color='red')
    
# Customize x-axis labels with month names
ax.set_xticks(data['month'])
ax.set_xticklabels([calendar.month_name[m] for m in data['month']], rotation=45)

# Customize the plot labels and title
ax.set_xlabel("Month")
#ax.set_ylabel("Median of Average Kilowatt per hour energy consumption")
ax.set_title("Monthly Energy Consumption, kwh per Industrial Customer Average (2013-2021)")
ax.ticklabel_format(style='plain', axis='y')

ax.legend()
plt.show()
# July is a peak month for industrial customers

In [ ]:
# now lets see how consumption changed over the years i.e. time-series trends

import calendar

grouped_data = df_years.groupby(['customerclass', 'year']).agg({'averagekwh': 'sum'})
# I use median as it's more robust to outliers as compared to average
grouped_data = grouped_data.reset_index()

# use all customer classes but industrial for which the consumption is very high and which we will plot separately
customer_classes = ['Elec- Residential','Elec- Commercial','Elec- Agricultural']
customer_class_labels = ['Residential Customers', 'Commercial Customers', 'Agricultural Customers']

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Iterate over each unique customer class and plot a line
for i,customer_class in enumerate(customer_classes):
    # Select data for the current customer class
    data = grouped_data[grouped_data['customerclass'] == customer_class]
    
    # Find the maximum average consumption (averagejwh) point in the data for the current customer class
    highest_point = data[data['averagekwh'] == data['averagekwh'].max()]
    
    # Plot the line for the current customer class
    ax.plot(data['year'], data['averagekwh'], label=customer_class_labels[i])
    
    # Plot a marker at the highest point
    ax.scatter(highest_point['year'], highest_point['averagekwh'], marker='o', color='red')
    
    # Customize the plot labels and title
ax.set_xlabel("Year")
#ax.set_ylabel("Median of Average Kilowatt per hour energy consumption")
ax.set_title("Annual Energy Consumption, kWh per Customer Average (2013-2021)")
ax.ticklabel_format(style='plain', axis='y')

ax.legend()
plt.show()
# Residential and Agricultural customers consumed the most energy in 2021 over the last 8 years 
# while it's the opposite for Commercial Clients who consumed the least amount of energy in 2021


In [ ]:
# now lets see how consumption changed over the years for industrial customers

grouped_data = df_years.groupby(['customerclass', 'year']).agg({'averagekwh': 'sum'})
# I use median as it's more robust to outliers as compared to average
grouped_data = grouped_data.reset_index()
customer_classes = ['Elec- Industrial']
customer_class_labels = ['Industrial Customers']

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 5))
data = grouped_data[grouped_data['customerclass'] == customer_class]
    
# Find the maximum average consumption (averagejwh) point 
highest_point = data[data['averagekwh'] == data['averagekwh'].max()]
    
ax.plot(data['year'], data['averagekwh'])
    
# Plot a marker at the highest point
ax.scatter(highest_point['year'], highest_point['averagekwh'], marker='o', color='red')
    
# Customize the plot labels and title
ax.set_xlabel("Year")
#ax.set_ylabel("Median of Average Kilowatt per hour energy consumption")
ax.set_title("Annual Energy Consumption, kWh per Industrial Customer Average (2013-2021)")
ax.ticklabel_format(style='plain', axis='y')

ax.legend()
plt.show()
# Residential and Agricultural customers consumed the most energy in 2021 over the last 9 years 
# while it's the opposite for Commercial Clients who consumed the least amount of energy in 2021
